In [1]:
# https://stackoverflow.com/questions/37159070/multiple-linear-regression-model-by-using-tensorflow
# https://donaldpinckney.com/books/pytorch/book/ch2-linreg/2018-03-21-multi-variable.html
# https://www.youtube.com/watch?v=Q4GNLhRtZNc
# https://atmamani.github.io/projects/ml/coursera-gd-multivariate-linear-regression/
# https://online.stat.psu.edu/stat462/sites/onlinecourses.science.psu.edu.stat462/files/05mlr/eq_matrix_notation/index.gif

# Multivariable Logistic Regression for matricies.
# target = flux1 + flux2 +... flux500 + b
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import pandas as pd
from numpy import exp

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [3]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename):
    data = np.matrix(pd.read_csv(filename).values)

    #we need to transpose data sets to be compatable with our weight vectors
    data_y = data[:, 0].transpose()
    data_y-=1 #binaryization of the categorical data

    data_x = data[:, 1:].transpose()
    data_x = normalize(data_x)
    return data_x, data_y

In [4]:
#Logistic Layer using a sigmoid function
def logistic_layer(y):
    y = np.array(y)
    y = 1 / (1 + exp(-y)) # sigmoid function
    y = y.ravel()
    return y

In [5]:
#Calculate an accuracy metric
def accuracy(predicted_y, true_y):
    true_y = np.array(true_y).ravel()
    counter = 0
    for i in range(len(true_y)):
        p_y = predicted_y[i]
        t_y = true_y[i]
        if (p_y>.5 and t_y == 1) or (p_y < .5 and t_y == 0):
            counter+=1
    counter = (counter/ len(true_y)) * 100
    return counter

In [6]:
training_data_x, training_data_y = features_and_labels("../archive/exoTrain.csv")

#set hyperparameters & variables
learning_rate = 0.003
epochs = 50
display_step = 5
n_samples = training_data_x.shape[1]
col_num = training_data_x.shape[0]

X = tf.placeholder(tf.float32, [col_num, n_samples])
Y = tf.placeholder(tf.float32, [1, n_samples]) #resulting dimenstion of W*X matmul

In [7]:
# We want the weight vector to correspond one to one with every column
W = tf.Variable(tf.zeros([1,col_num], dtype=np.float32), name="weight")
b = tf.Variable(tf.zeros([1, ], dtype=np.float32), name="bias")

#matrix multiplication requires outer dimension of W to be equal to be equal to the inner dimension of X: 
# (1,col_num) & (col_num, num_samples) - this is why we transpose X
pred = tf.matmul(W, X) + b # y′(x,A,b)=Ax+b linear matrix equation

error = tf.reduce_sum((pred-Y)**2) / (n_samples * 2) #MSE

In [8]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

session = tf.Session()
session.run(tf.global_variables_initializer())

loss_arr = []
acc_arr = []

for t in range(epochs):
    
    _, current_loss, current_W, current_b = session.run([optimizer, error, W, b], feed_dict={
        X: training_data_x,
        Y: training_data_y
    })

    if t % display_step == 0:
        print("t = %g, loss = %g, W = %s, b = %s" % (t, current_loss, str(current_W), str(current_b)))
    
    loss_arr.append(current_loss)
    acc_arr.append(accuracy(logistic_layer(current_W * training_data_x + current_b), training_data_y))
    
print("Optimization Finished!")

training_error = session.run(error, feed_dict={X: training_data_x, Y: training_data_y})
print("Training error=", training_error, "W=", session.run(W), "b=", session.run(b), '\n')

2021-11-04 19:58:57.312857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 19:58:57.422057: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 19:58:57.422761: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 19:58:57.424289: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

t = 0, loss = 0.00363672, W = [[0. 0. 0. ... 0. 0. 0.]], b = [0.]
t = 5, loss = 0.00356816, W = [[6.5006716e-05 5.8240956e-05 3.7853115e-05 ... 1.8715666e-05
  3.0330835e-05 3.8086691e-05]], b = [0.00011102]
t = 10, loss = 0.00352213, W = [[1.3444090e-04 1.1997453e-04 8.0120153e-05 ... 4.0680599e-05
  6.3859370e-05 7.9387159e-05]], b = [0.00022158]
t = 15, loss = 0.00348207, W = [[2.0233913e-04 1.7969179e-04 1.2091576e-04 ... 6.2688814e-05
  9.7530792e-05 1.2125886e-04]], b = [0.00033039]
t = 20, loss = 0.00344504, W = [[2.6741525e-04 2.3627648e-04 1.5902193e-04 ... 8.4209198e-05
  1.3077853e-04 1.6306229e-04]], b = [0.00043767]
t = 25, loss = 0.00341012, W = [[0.00032955 0.00028971 0.00019438 ... 0.00010513 0.00016345 0.00020453]], b = [0.00054352]
t = 30, loss = 0.00337687, W = [[0.00038889 0.0003402  0.0002272  ... 0.00012544 0.00019551 0.00024554]], b = [0.00064801]
t = 35, loss = 0.00334507, W = [[0.00044564 0.00038798 0.00025771 ... 0.00014518 0.00022697 0.00028605]], b = [0.0007

In [9]:
test_x, test_y = features_and_labels("../archive/exoTest.csv")

predicted_y = session.run(W) * test_x + session.run(b)
predicted_y = logistic_layer(predicted_y)

print("Accuracy percentage: ", accuracy(predicted_y, test_y), "%")

Accuracy percentage:  60.526315789473685 %


In [20]:
with open('tf-models/multi-lr.npy', 'wb') as f:
    np.save(f, session.run(W))
    np.save(f, session.run(b))
    np.save(f, loss_arr)
    np.save(f, acc_arr)
